You can download the `requirements.txt` for this course from the workspace of this lab. `File --> Open...`

# L2: Create Agents to Research and Write an Article

In this lesson, you will be introduced to the foundational concepts of multi-agent systems and get an overview of the crewAI framework.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [2]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import from the crewAI libray.

In [3]:
from crewai import Agent, Task, Crew

- As a LLM for your agents, you'll be using OpenAI's `gpt-3.5-turbo`.

**Optional Note:** crewAI also allow other popular models to be used as a LLM for your Agents. You can see some of the examples at the [bottom of the notebook](#1).

In [4]:
import os
from utils import get_openai_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'

## Creating Agents

- Define your Agents, and provide them a `role`, `goal` and `backstory`.
- It has been seen that LLMs perform better when they are role playing.

### Agent: Planner

**Note**: The benefit of using _multiple strings_ :
```Python
varname = "line 1 of text"
          "line 2 of text"
```

versus the _triple quote docstring_:
```Python
varname = """line 1 of text
             line 2 of text
          """
```
is that it can avoid adding those whitespaces and newline characters, making it better formatted to be passed to the LLM.

In [5]:
planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic.",
    allow_delegation=False,
	verbose=True
)

### Agent: Writer

In [6]:
writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    allow_delegation=False,
    verbose=True
)

### Agent: Editor

In [7]:
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    allow_delegation=False,
    verbose=True
)

## Creating Tasks

- Define your Tasks, and provide them a `description`, `expected_output` and `agent`.

### Task: Plan

In [8]:
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,
)

### Task: Write

In [9]:
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		"3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    agent=writer,
)

### Task: Edit

In [10]:
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor
)

## Creating the Crew

- Create your crew of Agents
- Pass the tasks to be performed by those agents.
    - **Note**: *For this simple example*, the tasks will be performed sequentially (i.e they are dependent on each other), so the _order_ of the task in the list _matters_.
- `verbose=2` allows you to see all the logs of the execution. 

In [11]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    verbose=2
)

## Running the Crew

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [16]:
result = crew.kickoff(inputs={"topic": "Artificial Intelligence"})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on Artificial Intelligence.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer:

Content Plan: 

Title: The Future of Artificial Intelligence: Trends, Key Players, and News

Introduction:
- Brief overview of Artificial Intelligence (AI) and its impact on various industries
- Importance of staying updated on the latest trends and key players in the AI space

Key Points:
1. Latest Trends in Artificial Intelligence:
- Machine learning advancements
- Natural language processing developments
- Autonomous vehicles and robotics
- AI ethics and bias concerns

2. Key Players in the Artificial Inte

I now can give a great answer.

Final Answer:

# The Future of Artificial Intelligence: Trends, Key Players, and News

## Introduction

Artificial Intelligence (AI) has revolutionized various industries, from healthcare to finance, by providing innovative solutions and enhancing efficiency. As AI continues to evolve rapidly, it is crucial for individuals to stay updated on the latest trends and key players in the AI space. This blog post aims to explore the current landscape of AI, highlighting the trends shaping the industry, the key players driving innovation, and the noteworthy news that is shaping the future of AI.

## Latest Trends in Artificial Intelligence

One of the most significant trends in Artificial Intelligence is the advancements in machine learning. Machine learning algorithms are becoming increasingly sophisticated, enabling AI systems to learn from data and improve their performance over time. Additionally, natural language processing has seen significant developments

- Display the results of your execution as markdown in the notebook.

In [17]:
from IPython.display import Markdown
Markdown(result)

# The Future of Artificial Intelligence: Trends, Key Players, and News

## Introduction

Artificial Intelligence (AI) has revolutionized various industries, from healthcare to finance, by providing innovative solutions and enhancing efficiency. As AI continues to evolve rapidly, it is crucial for individuals to stay updated on the latest trends and key players in the AI space. This blog post aims to explore the current landscape of AI, highlighting the trends shaping the industry, the key players driving innovation, and the noteworthy news that is shaping the future of AI.

## Latest Trends in Artificial Intelligence

One of the most significant trends in Artificial Intelligence is the advancements in machine learning. Machine learning algorithms are becoming increasingly sophisticated, enabling AI systems to learn from data and improve their performance over time. Additionally, natural language processing has seen significant developments, with AI-powered language models such as GPT-3 achieving remarkable capabilities in understanding and generating human-like text. The rise of autonomous vehicles and robotics is also a key trend in AI, with companies like Tesla and Boston Dynamics pushing the boundaries of what is possible with AI-driven technologies. However, along with these advancements come concerns about AI ethics and bias, highlighting the importance of responsible AI development and deployment.

## Key Players in the Artificial Intelligence Industry

Several key players are driving innovation in the AI industry, including Google AI, IBM Watson, Amazon Web Services (AWS), Microsoft AI, and OpenAI. These companies are at the forefront of AI research and development, creating cutting-edge solutions that are shaping the future of AI. Google AI, for example, is known for its work in machine learning and natural language processing, while IBM Watson is a leader in AI-powered healthcare solutions. Amazon Web Services provides cloud-based AI services to businesses worldwide, and Microsoft AI focuses on democratizing AI by making it accessible to all. OpenAI, on the other hand, is known for its research in artificial general intelligence, pushing the boundaries of what AI can achieve.

## Noteworthy News in Artificial Intelligence

Recent breakthroughs in AI research have been making headlines, from advancements in computer vision to the development of AI-driven drug discovery platforms. Industry partnerships and collaborations are also shaping the AI landscape, with companies coming together to develop AI solutions for specific industries and use cases. Regulatory updates in AI technology are also important to monitor, as governments around the world are working to establish guidelines and frameworks for the ethical use of AI. Staying informed on these developments is crucial for anyone looking to leverage AI in their business or research.

## Conclusion

In conclusion, Artificial Intelligence is a rapidly evolving field with endless possibilities for innovation and growth. By staying informed on the latest trends, key players, and news in the AI industry, individuals can make informed decisions and stay ahead of the curve. Whether you are a tech enthusiast, a business professional, a student, or a policy maker, understanding the complex concepts and advancements in AI is essential for navigating the future of technology. Subscribe to our newsletter for regular updates on AI trends and explore the resources provided for further reading on AI news and developments. Let's embrace the future of Artificial Intelligence together and shape a world powered by innovation and technology.

## Try it Yourself

- Pass in a topic of your choice and see what the agents come up with!

In [18]:
topic = "Multi Agent Systems"
result = crew.kickoff(inputs={"topic": topic})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on Multi Agent Systems.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer:

Content Plan: Multi Agent Systems

Outline:
I. Introduction
- Definition of Multi Agent Systems
- Importance and applications in various fields
II. Latest Trends in Multi Agent Systems
- Adoption of machine learning and AI in MAS
- Integration of MAS in smart cities and IoT
III. Key Players in Multi Agent Systems
- Companies leading in MAS development
- Research institutions and universities focusing on MAS
IV. Noteworthy News on Multi Agent Systems
- Recent advancements and breakthroughs in MAS
- Impact of MAS o

I now can give a great answer

Final Answer:
# Exploring the Future of Multi Agent Systems

## Introduction

Multi Agent Systems (MAS) refer to a group of intelligent agents that interact with each other to achieve a common goal. These systems have gained significant importance and find applications in various fields such as robotics, autonomous vehicles, smart cities, and Internet of Things (IoT). MAS enable decentralized decision-making and coordination among agents, leading to efficient and scalable solutions in complex environments.

## Latest Trends in Multi Agent Systems

One of the latest trends in MAS is the adoption of machine learning and artificial intelligence (AI) techniques. By integrating these technologies, MAS can enhance their decision-making capabilities and adapt to dynamic environments. Moreover, MAS are increasingly being utilized in smart cities and IoT applications to optimize resource allocation, traffic management, and energy efficiency. The integration of MAS

In [19]:
Markdown(result)

# Exploring the Future of Multi Agent Systems

## Introduction

Multi Agent Systems (MAS) refer to a group of intelligent agents that interact with each other to achieve a common goal. These systems have gained significant importance and find applications in various fields such as robotics, autonomous vehicles, smart cities, and Internet of Things (IoT). MAS enable decentralized decision-making and coordination among agents, leading to efficient and scalable solutions in complex environments.

## Latest Trends in Multi Agent Systems

One of the latest trends in MAS is the adoption of machine learning and artificial intelligence (AI) techniques. By integrating these technologies, MAS can enhance their decision-making capabilities and adapt to dynamic environments. Moreover, MAS are increasingly being utilized in smart cities and IoT applications to optimize resource allocation, traffic management, and energy efficiency. The integration of MAS in these domains is shaping the future of urban planning and infrastructure development.

## Key Players in Multi Agent Systems

Several companies are at the forefront of MAS development, including Google, IBM, and Amazon. These tech giants are investing in research and development to push the boundaries of MAS capabilities and applications. Additionally, research institutions and universities such as MIT, Stanford, and Carnegie Mellon University are focusing on advancing MAS through cutting-edge research and innovation. Their contributions are shaping the landscape of MAS and driving progress in the field.

## Noteworthy News on Multi Agent Systems

Recent advancements in MAS include breakthroughs in swarm intelligence, where large groups of agents collaborate to solve complex problems. This approach mimics the behavior of natural systems such as ant colonies and flocking birds, leading to innovative solutions in optimization and decision-making. The impact of MAS on industries and society is profound, with applications ranging from supply chain management to healthcare delivery. As MAS continue to evolve, the possibilities for innovation and disruption are endless.

## Conclusion

In conclusion, Multi Agent Systems are revolutionizing the way we approach complex problems and optimize decision-making processes. By staying informed on the latest trends, key players, and noteworthy news in MAS, professionals in AI, machine learning, and robotics can leverage this technology to drive innovation and create new opportunities. As businesses look to implement MAS solutions, it is crucial to stay updated on advancements and breakthroughs in the field. Subscribe to our newsletter for exclusive content and insights to stay ahead of the curve in the exciting world of Multi Agent Systems.

<a name='1'></a>
 ## Other Popular Models as LLM for your Agents

#### Hugging Face (HuggingFaceHub endpoint)

```Python
from langchain_community.llms import HuggingFaceHub

llm = HuggingFaceHub(
    repo_id="HuggingFaceH4/zephyr-7b-beta",
    huggingfacehub_api_token="<HF_TOKEN_HERE>",
    task="text-generation",
)

### you will pass "llm" to your agent function
```

#### Mistral API

```Python
OPENAI_API_KEY=your-mistral-api-key
OPENAI_API_BASE=https://api.mistral.ai/v1
OPENAI_MODEL_NAME="mistral-small"
```

#### Cohere

```Python
from langchain_community.chat_models import ChatCohere
# Initialize language model
os.environ["COHERE_API_KEY"] = "your-cohere-api-key"
llm = ChatCohere()

### you will pass "llm" to your agent function
```

### For using Llama locally with Ollama and more, checkout the crewAI documentation on [Connecting to any LLM](https://docs.crewai.com/how-to/LLM-Connections/).